In [1]:
import numpy as np
import pandas as pd
import json
import requests
import time
import itertools
import sqlite3
import operator

In [2]:
def patent_date(patent):
    try:
        return patent['patent_date']
    except:
        return ''

def patent_num_cited_by_us_patents(patent):
    try:
        return patent['patent_num_cited_by_us_patents']
    except:
        return ''

def patent_num_us_patent_citations(patent):
    try:
        return patent['patent_num_us_patent_citations']
    except:
        return ''

def patent_date(patent):
    try:
        return patent['patent_date']
    except:
        return ''
    
def patent_number(patent):
    try:
        return patent['patent_number']
    except:
        return ''

def patent_title(patent):
    try:
        return patent['patent_title']
    except:
        return ''

def patent_type(patent):
    try:
        return patent['patent_type']
    except:
        return ''

def patent_year(patent):
    try:
        return patent['patent_year']
    except:
        return ''
    
def patent_abstract(patent):
    try:
        return patent['patent_abstract']
    except:
        return ''

def cpc_category(patent):
    try:
        return patent['cpcs'][0]['cpc_category']
    except:
        return ''
    
def cpc_section_id(patent):
    try:
        return patent['cpcs'][0]['cpc_section_id']
    except:
        return ''

def assignee_city(patent):
    try:
        return patent['assignees'][0]['assignee_city']
    except:
        return ''

def assignee_country(patent):
    try:
        return patent['assignees'][0]['assignee_country']
    except:
        return ''

def assignee_latitude(patent):
    try:
        return patent['assignees'][0]['assignee_latitude']
    except:
        return ''

def assignee_longitude(patent):
    try:
        return patent['assignees'][0]['assignee_longitude']
    except:
        return ''

def assignee_organization(patent):
    try:
        return patent['assignees'][0]['assignee_organization']
    except:
        return ''

def assignee_state(patent):
    try:
        return patent['assignees'][0]['assignee_state']
    except:
        return ''

def assignee_type(patent):
    try:
        return patent['assignees'][0]['assignee_type']
    except:
        return ''

def cited_patents(patent):
    if patent_num_us_patent_citations(patent) != '0':
        patentnum = patent_number(patent)
        rows = []
        for dictt in patent['cited_patents']:
            if dictt['cited_patent_number'] == None:
                continue
            else:
                rows.append((patentnum,patent_num_us_patent_citations(patent),dictt['cited_patent_number'],dictt['cited_patent_title']))
        return rows
    else:
        return ''

def citedby_patents(patent):
    if patent_num_cited_by_us_patents(patent) != '0':
        patentnum = patent_number(patent)
        rows = []
        for dictt in patent['citedby_patents']:
            if dictt['citedby_patent_number'] == None:
                continue
            else:
                rows.append((patentnum,patent_num_cited_by_us_patents(patent),dictt['citedby_patent_number'],dictt['citedby_patent_title']))
        return rows
    else:
        return ''

def first_inventor_name(patent):
    try:
        inventors= patent['inventors']
    except:
        return ''
    inventors.sort(key=operator.itemgetter('inventor_sequence'))
    if len(inventors) >0 and None not in inventors[0].values(): 
        if len(inventors[0]['inventor_first_name']) >0:
            return inventors[0]['inventor_first_name'] + ' ' + inventors[0]['inventor_last_name']
        else:
            return ''
    else:
        return ''

def coinventors(patent):
    try:
        inventors = patent['inventors']
    except:
        return ''
    inventors.sort(key=operator.itemgetter('inventor_sequence'))
    coinventors = ''
    
    if len(inventors)>0:
        for i in range(1,len(inventors)):
            if None not in inventors[i].values():
                if i == len(inventors) - 1:
                    coinventors = coinventors + inventors[i]['inventor_first_name'] + ' ' + inventors[i]['inventor_last_name']
                else:
                    coinventors = coinventors + inventors[i]['inventor_first_name'] + ' ' + inventors[i]['inventor_last_name'] + ', '
            else:
                return ''
        return coinventors
    else:
        return ''

def cpc_category(patent):
    try:
        return patent['cpcs'][0]['cpc_category']
    except:
        return ''

def cpc_section_id(patent):
    try:
        return patent['cpcs'][0]['cpc_section_id']
    except:
        return ''


In [3]:
sqlite_file = 'Patent-Project-Rev1.sqlite'
conn = sqlite3.connect(sqlite_file)
c = conn.cursor()

In [4]:
conn.commit()

In [33]:
#c.execute('''CREATE TABLE Patents (patent_number TEXT PRIMARY KEY,
patent_date TEXT,
patent_type TEXT,
patent_year INTEGER,
patent_title TEXT,
patent_abstract TEXT,
first_inventor TEXT,
co_inventors TEXT,
cpc_category TEXT,
cpc_section_id TEXT,
assignee_city TEXT,
assignee_country TEXT,
assignee_organization TEXT,
assignee_state TEXT,
assignee_type TEXT,
no_patents_citeby INTEGER,
no_patents_cited INTEGER
);''')

In [34]:
#c.execute('''CREATE TABLE PatentsCited (patent_number TEXT,
no_patents_cited INTEGER,
cited_patent_number TEXT,
cited_patent_title TEXT
);''')

In [13]:
request_lapse = 1
page_num = 1
per_page = 2000
patents_json = []
num_patents_read = 0
count = 0
total_patents = 10000

In [14]:
while (num_patents_read < total_patents):
    if count == 0:
        url ='http://www.patentsview.org/api/patents/query?q={"_and":[{"_gte":{"patent_date":"2010-01-01"}},{"_lte":{"patent_date":"2010-12-31"}},{"assignee_country":"US"}]}'
        test_cit = requests.get(url)
        total_patents = json.loads(test_cit.content)['total_patent_count']
        print "Total number of patents to read {}".format(total_patents)
        count = count + 1
    else:
        url ='http://www.patentsview.org/api/patents/query?q={"_and":[{"_gte":{"patent_date":"2010-01-01"}},{"_lte":{"patent_date":"2010-12-31"}},{"assignee_country":"US"}]}&f=["patent_number","patent_date","patent_title","patent_year","inventor_first_name","inventor_last_name","patent_abstract","cited_patent_number","cited_patent_title","citedby_patent_number","citedby_patent_title","assignee_type","assignee_organization","assignee_country","assignee_state","assignee_city","patent_type","cpc_category","cpc_section_id","assignee_latitude","assignee_longitude","patent_num_cited_by_us_patents","patent_num_us_patent_citations","inventor_sequence"]'
        url = url + '&o=' + '{{"page":{},"per_page":{}}}'.format(page_num,per_page)
        test_cit = requests.get(url)
        patents_json = json.loads(test_cit.content)['patents']
        #flattened_list  = list(itertools.chain(*patents_json))
        
        for patent in patents_json:
            c.execute('INSERT INTO Patents VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)',
             (patent_number(patent),patent_date(patent),patent_type(patent),patent_year(patent),
              patent_title(patent),patent_abstract(patent),first_inventor_name(patent),coinventors(patent),
             cpc_category(patent),cpc_section_id(patent),assignee_city(patent),assignee_country(patent),
             assignee_organization(patent),assignee_state(patent),assignee_type(patent),
             patent_num_cited_by_us_patents(patent),patent_num_us_patent_citations(patent)))
        
            for citedpat in cited_patents(patent):
                c.execute('INSERT INTO PatentsCited VALUES (?,?,?,?)',
                         (citedpat[0],citedpat[1],citedpat[2],citedpat[3]))
                
        page_num = page_num + 1
        num_patents_read = num_patents_read + len(patents_json)
        print "Read {} patents from PatentsView.org so far".format(num_patents_read)
        count = count + 1
        time.sleep(request_lapse)

Total number of patents to read 100000
Read 2000 patents from PatentsView.org so far
Read 4000 patents from PatentsView.org so far
Read 6000 patents from PatentsView.org so far
Read 8000 patents from PatentsView.org so far
Read 10000 patents from PatentsView.org so far
Read 12000 patents from PatentsView.org so far
Read 14000 patents from PatentsView.org so far
Read 16000 patents from PatentsView.org so far
Read 18000 patents from PatentsView.org so far
Read 20000 patents from PatentsView.org so far
Read 22000 patents from PatentsView.org so far
Read 24000 patents from PatentsView.org so far
Read 26000 patents from PatentsView.org so far
Read 28000 patents from PatentsView.org so far
Read 30000 patents from PatentsView.org so far
Read 32000 patents from PatentsView.org so far
Read 34000 patents from PatentsView.org so far
Read 36000 patents from PatentsView.org so far
Read 38000 patents from PatentsView.org so far
Read 40000 patents from PatentsView.org so far
Read 42000 patents from P

In [15]:
conn.commit()

In [134]:
conn.close()

In [45]:
#below is just testing work

In [116]:
patents_json[22]

{u'assignees': [{u'assignee_city': u'Rancho Santa Margarita',
   u'assignee_country': u'US',
   u'assignee_latitude': u'33.6408',
   u'assignee_longitude': u'-117.602',
   u'assignee_organization': u'Crucible Intellectual Property, LLC',
   u'assignee_state': u'CA',
   u'assignee_type': u''}],
 u'cited_patents': [{u'cited_patent_number': None,
   u'cited_patent_title': None},
  {u'cited_patent_number': u'3948613',
   u'cited_patent_title': u'Process for applying a protective wear surface to a wear part'},
  {u'cited_patent_number': u'3970445',
   u'cited_patent_title': u'Wear-resistant alloy, and method of making same'},
  {u'cited_patent_number': u'3986867',
   u'cited_patent_title': u'Iron-chromium series amorphous alloys'},
  {u'cited_patent_number': u'3986892',
   u'cited_patent_title': u'Porous cobalt electrodes for alkaline accumulators and hybrid cell therewith and air electrode'},
  {u'cited_patent_number': u'4024902',
   u'cited_patent_title': u'Method of forming metal tungste

In [98]:
count = 0
for patent in patents_json:
    invent = first_inventor_name(patent)
    count = count + 1

In [99]:
count

2000

In [51]:
patents_json[277]['inventors'][0]

{u'inventor_first_name': None,
 u'inventor_last_name': None,
 u'inventor_sequence': None}

In [54]:
if None not in patents_json[277]['inventors'][0].values():
    print 'yeah'

In [91]:
p = patents_json[278]
inventors = p['inventors']
print inventors
print len(inventors)
if len(inventors) >0 and None not in inventors[0].values():
    print 'yeah'

[{u'inventor_sequence': u'0', u'inventor_first_name': u'Hugo L.', u'inventor_last_name': u'Zorrilla'}]
1
yeah


In [84]:
None in [d.values() for d in inventors]

False

In [76]:
a = [{u'inventor_sequence': u'1', u'inventor_first_name': None, u'inventor_last_name': u'Frank'}, {u'inventor_sequence': u'0', u'inventor_first_name': u'Sandeep H.', u'inventor_last_name': u'Krishnamurthy'}]
None in inventors[:].values()

AttributeError: 'list' object has no attribute 'values'

In [126]:
numrows = c.execute("SELECT COUNT (*) FROM Patents")
numrows.fetchall()

[(539947,)]

In [131]:
patents_json[0]

IndexError: list index out of range

In [18]:
findrows = conn.cursor()
testrows = findrows.execute("SELECT * FROM Patents WHERE patent_number='9693497'")
testrows.fetchall()

[(u'9693497',
  u'2017-07-04',
  u'utility',
  2017,
  u'Mounting bracket for agricultural row unit',
  u'A row unit for an agricultural planter is provided. The row unit may include a body operably coupled to the agricultural planter, a bracket secured to the body, and a chassis pivotably coupled to the bracket. The bracket may include a top wall and opposing sidewalls extending downwardly from the top wall. The chassis may be coupled to the bracket beneath the top wall of the bracket.',
  u'Paul E. Schaffert',
  u'',
  u'inventional',
  u'A',
  u'Indianola',
  u'US',
  u'SCHAFFERT MANUFACTURING COMPANY, INC.',
  u'IA',
  u'',
  0,
  78)]

In [19]:
conn.close()

In [100]:
def first_inventor_name(patent):
    try:
        inventors= patent['inventors']
    except:
        return ''
    inventors.sort(key=operator.itemgetter('inventor_sequence'))
    if len(inventors) >0 and None not in inventors[0].values(): 
        if len(inventors[0]['inventor_first_name']) >0:
            return inventors[0]['inventor_first_name'] + ' ' + inventors[0]['inventor_last_name']
        else:
            return ''
    else:
        return ''

def coinventors(patent):
    try:
        inventors = patent['inventors']
    except:
        return ''
    inventors.sort(key=operator.itemgetter('inventor_sequence'))
    coinventors = ''
    
    if len(inventors)>0:
        for i in range(1,len(inventors)):
            if None not in inventors[i].values():
                if i == len(inventors) - 1:
                    coinventors = coinventors + inventors[i]['inventor_first_name'] + ' ' + inventors[i]['inventor_last_name']
                else:
                    coinventors = coinventors + inventors[i]['inventor_first_name'] + ' ' + inventors[i]['inventor_last_name'] + ', '
            else:
                return ''
        return coinventors
    else:
        return ''

In [101]:
count = 0
for patent in patents_json:
    invent = coinventors(patent)
    count = count + 1

In [102]:
count

2000